# Selection of synonymous pairs

In [12]:
import pickle
from tqdm import tqdm
import os

import scipy.sparse as sp
import numpy as np
import pandas as pd

In [13]:
pos = 'A'
suffix = 'doubNorm_cosine/'
SEMCHANGE_FOLDER = './semantic_change_measures/'+suffix

DECADES = list(range(1890,2000,10))

if pos.lower() in ['a','adj','adjective']:
    pos = 'A'
    with open('./words/adjs_list.pickle','rb') as f:
        word_list = pickle.load(f)
elif pos.lower() in ['n','noun']:
    pos = 'N'
    with open('./words/nouns_list.pickle','rb') as f:
        word_list = pickle.load(f)
elif pos.lower() in ['v','verb']:
    pos = 'V'
    with open('./words/verbs_list.pickle','rb') as f:
        word_list = pickle.load(f)
else:
    raise ValueError('Invalid POS tag value : ' + pos)

word2ind = {word : i for i,word in enumerate(word_list)}
with open('./words/fernald_synonyms.pickle','rb') as f:
    syns_dict = pickle.load(f)[pos]
semchanges = pd.read_csv(SEMCHANGE_FOLDER+pos+'.csv',sep='\t',index_col='words')

## Original Criterion : both words changed

In [20]:
target_changes = semchanges['1990']
threshold = target_changes.mean()
syns_pairs = []
for head, syns in syns_dict.items():
    if target_changes[head] >= threshold:
        for syn in syns:
            if target_changes[syn] >= threshold:
                syns_pairs.append( (head,syn) )

In [9]:
# with open(f'./words/fernald_syn_pairs_{pos}.pickle','wb') as f:
#     pickle.dump(file=f,obj=syns_pairs)

In [21]:
len(syns_pairs)

271

In [22]:
len(syns_dict)

143

In [23]:
sum([len(syns) for syns in syns_dict.values()])

1450

In [24]:
len(target_changes)

6475

## New Criterion : at least one changed

In [14]:
target_changes = semchanges['1990']
threshold = target_changes.mean() #+ target_changes.std()
syns_pairs = []
for head, syns in syns_dict.items():
    for syn in syns:
        if target_changes[syn] >= threshold or target_changes[head] >= threshold:
            syns_pairs.append( (head,syn) )

In [15]:
len(syns_pairs)

1096

In [18]:
with open(f'./words/synonym_pairs/{suffix}/fernald_syn_pairs_{pos}_NEWCRIT.pickle','wb') as f:
    pickle.dump(file=f,obj=syns_pairs)